In [1]:
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline
import gradio as gr

In [2]:
# Load the summarization model
summarizer = pipeline(task='summarization', model='sshleifer/distilbart-cnn-12-6')
#Load the NER model
named_entity_recognizer= pipeline(task = 'ner', model = 'dslim/bert-base-NER')

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# NER helper functions
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens
def ner(input):
    output = named_entity_recognizer(input)
    merged_tokens = merge_tokens(output)
    return {'text': input, 'entities': merged_tokens}

NER = gr.Interface(
    fn = ner,
    inputs = [gr.Textbox(label = "Text to find entities", lines = 3)],
    outputs = [gr.HighlightedText(label = 'Text with entities')],
    allow_flagging = 'never',
    examples=[
        "My name is Nabi, I'm building NER Application",
    "My name is Emon, I live in Rajshahi and study at RUET"
    ]
)

# Add Markdown content
markdown_content_ner = gr.Markdown(
    """
    <div style='text-align: center; font-family: "Times New Roman";'>
        <h1 style='color: #FF6347;'>Named Entity Recognition APP</h1>
        <h3 style='color: #4682B4;'>Model: dslim/bert-base-NER</h3>
        <h3 style='color: #32CD32;'>Made By: Md. Mahmudun Nabi</h3>
    </div>
    """
)

# Combine the Markdown content and the demo interface
ner_with_markdown = gr.Blocks()
with ner_with_markdown:
    markdown_content_ner.render()
    NER.render()

In [4]:
# Function to summarize input text
def summarize(input):
    output = summarizer(input)
    return output[0]['summary_text']

# Create the Gradio interface
SUMMARIZER = gr.Interface(
    fn=summarize,
    inputs=[gr.Textbox(label='Text to summarize', lines=6)],
    outputs=[gr.Textbox(label='Result', lines=3)],
)

# Add Markdown content
markdown_content_summarizer = gr.Markdown(
    """
    <div style='text-align: center; font-family: "Times New Roman";'>
        <h1 style='color: #FF6347;'>Text Summarization with DistilBART-CNN</h1>
        <h3 style='color: #4682B4;'>Model: sshleifer/distilbart-cnn-12-6</h3>
        <h3 style='color: #32CD32;'>Made By: Md. Mahmudun Nabi</h3>
    </div>
    """
)

# Combine the Markdown content and the demo interface
summarizer_with_markdown = gr.Blocks()
with summarizer_with_markdown:
    markdown_content_summarizer.render()
    SUMMARIZER.render()

In [5]:
# Combine both the app
demo = gr.Blocks()
with demo:
    gr.TabbedInterface(
        [ner_with_markdown, summarizer_with_markdown],
        ['Named Entity Recognition', 'Text Summarization']
    )

In [6]:
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
